In [1]:
import pandas as pd
import datetime
import sys
sys.path.append('./utils')
from utils import *
from process import * 
from add_list_master import *
import japanize_matplotlib

In [2]:
# データ保存先
data_folda = '../data/'

target_column="target6"
make_genre_list=True

train_name="train_with_master_structured_main_medias.pkl"
test_name="test3_with_master_structured_main_medias.pkl"

In [3]:
train_data = pd.read_pickle(data_folda+train_name)
test_data = pd.read_pickle(data_folda+test_name)

In [4]:
# object型のカラムをリスト化する関数
def get_object_column_list(df):
    object_list = []
    for column in df.columns:
        if df[column].dtype == "object":
            object_list.append(column)
    return object_list

object_list = get_object_column_list(train_data)
object_list

['restaurant_id',
 'name',
 'prefecture_name',
 'head_branch',
 'created_at',
 'genre_first_name',
 'genre_second_name',
 'genre_third_name',
 'overview.reviewCount',
 'general_rank_GOOGLE',
 'infos.familiar.users.genge',
 'infos.familiar.stars',
 'general_rank_RETTY',
 'infos.reserveEntireStore',
 'infos.counterSeats.value',
 'infos.privateRoom.value',
 'infos.updateInfo.lastReviewDate',
 'general_rank_HOTPEPPER',
 'infos.couponCount',
 'photos.allCount',
 'photos.appearanceCount',
 'photos.drinkCount',
 'photos.foodCount',
 'photos.otherCount',
 'photos.postCount',
 'menus.courseListCount',
 'url',
 'aggregateRating.ratingCount',
 'aggregateRating.ratingValue',
 'aggregateRating.reviewCount',
 'aggregateRating.bestRating',
 'aggregateRating.worstRating',
 'interactionCount.went',
 'interactionCount.wanttogo',
 'paymentAccepted',
 'general_rank_STRUCTURED',
 'hostname',
 'general_rank_TRIPADVISOR',
 'infos.aggregateRating_TRIPADVISOR',
 'infos.reviewCount',
 'infos.qa_count',
 'genera

In [5]:
# object型を表示

train_data = train_data[object_list+["base_date"]]
train_data_filter = train_data[train_data["base_date"].dt.strftime('%Y%m')=="202212"]

In [6]:
# eval_period=True の際の target3=0,1 毎の genre_first_name のランキング
#train_data[(train_data["eval_period"]==True)].groupby(["target3","genre_first_name"]).size().unstack().fillna(0).astype(int)

* レストランごとにジャンルをダミー化して保存する 

In [7]:

genre0=train_data["genre_first_name"].unique()
genre1=train_data["genre_second_name"].unique()
genre2=train_data["genre_third_name"].unique()

genre_list = list(set(list(genre0)+list(genre1)+list(genre2)))
genre_list = pd.Series(genre_list)
genre_list=genre_list.dropna()

genre_list=genre_list.reset_index(drop=True)

In [8]:
train_data_filter = train_data[train_data["base_date"].dt.strftime('%Y%m')=="202212"]

# train_data を base_date=2022年12月31日に絞ってダミー化
glist1 = [pd.DataFrame() for i in range(len(genre_list))]       
glist2 = [pd.DataFrame() for i in range(len(genre_list))]       
glist3 = [pd.DataFrame() for i in range(len(genre_list))]       

# 進行状況を表示
for i in tqdm(range(len(genre_list))):
    glist1[i]=train_data_filter["genre_first_name"].str.contains(genre_list[i]).replace('',0).astype(int)
    glist2[i]=train_data_filter["genre_second_name"].str.contains(genre_list[i]).replace('',np.nan).astype(float)
    glist3[i]=train_data_filter["genre_third_name"].str.contains(genre_list[i]).replace('',np.nan).astype(float)
    

glist1_df = pd.concat(glist1,axis=1)
glist1_df.columns=genre_list
glist2_df = pd.concat(glist2,axis=1)
glist2_df.columns=genre_list
glist3_df = pd.concat(glist3,axis=1)
glist3_df.columns=genre_list

glist_df = 3*glist1_df+2*glist2_df+glist3_df
glist_df.fillna(0,inplace=True)

train_data_restaurant_id = train_data[train_data["base_date"].dt.strftime('%Y%m')=="202212"][["restaurant_id","base_date"]]
train_data_restaurant_id = pd.concat([train_data_restaurant_id,glist_df],axis=1)
train_data_restaurant_id.to_pickle(data_folda+"category_dummies.pkl")



  0%|          | 0/220 [00:00<?, ?it/s]

* 必要な時に結合する

,restaurant_id,base_date,豚丼,ハワイ料理,薬膳,日本酒バー,バル,スイーツ,社員食堂,ロシア料理,...,ジューススタンド,カリフォルニア料理,惣菜・デリ,居酒屋,麺類,ホットドッグ,とんかつ,弁当,パキスタン料理,洋食
0,00000528-ea7b-4364-bb76-3d46fba520ef,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
12,00002990-5cde-49f2-a2e1-baca20c61fdc,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
24,00003d54-0a2a-43f5-b945-5ae9b0314942,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
36,00003ef6-4867-409a-896c-566134666192,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
48,00005256-746d-467d-bf65-82686b244108,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10503708,ffff88ea-fd34-46f8-a749-001a2c59cdba,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10503720,ffff9349-bd6a-4c56-bb06-c1c11af3b736,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10503732,ffffbb36-5a31-4f02-b1e5-5883dfc16f09,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10503744,ffffdb1a-05bf-4227-8ea8-cf5bb0b63a43,2022-12-31,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
# glist1, gliist2, glist3 を合計して、genre_ジャンル名 として新たなカラムを作成
#glist=[glist1,glist2,glist3]
#augumented_data_train=pd.concat([train_data]+glist1,axis=1)

In [10]:
# genre_first_name, genre_second_name, genre_third_name それぞれをダミー化したものを合算
# pd.get_dummies(train_data, columns=["genre_first_name","genre_second_name","genre_third_name"])